# Reshaping and Merging DataFrames

## In this section, we'll cover

* Learn about data reshaping
* What wide and long data formats are, and how to convert between them with `pivot` and `melt`
* How to merge data tables together with `concat` and `merge`

## What is Data Reshaping?

Real world data comes in all shapes and sizes, and it's not uncommon to encounter data that's poorly organized in ways that makes it difficult to work with.  Therefore, learning how to reorganize data is a critical skill for anyone working with data.  This process is generally referred to as data reshaping.

While there are all kinds of strange and interesting ways in which data sets can formatted, there are some common organization patterns you'll often encounter.  Two that we'll discuss here are *wide* and *long* forms of tabular data.

Let's first illustrate this with an example of a data table in long format -- the Gapminder dataset (from https://www.gapminder.org, tidied by Jennifer Bryan https://github.com/jennybc).  This is a very interesting data set that looks at life expectancy and per capita GDP for countries across time.

In [ ]:
import pandas as pd

# This is actually a tab separated file (not comma); still use read_csv, but specify the tab separator
gapminder = pd.read_csv("https://raw.githubusercontent.com/jennybc/gapminder/master/inst/extdata/gapminder.tsv", sep="\t")
gapminder.head(10)

The term *long* refers to the notion that this data long (or tall) in terms of the number of rows vs. the number of columns (thought this does not need to be strictly true). If we were to add additional measurements to this table (e.g. data for a new country or a new year), it would get even longer because we'd need to add additional rows.

## Reshaping Long Data to Wide Data

In the table above, measurements for each country have been made across time.  In this case, there is one row for each country and year combination such that each country is repeated across multiple rows (one for each year), and each year is also repeated across multiple rows (one for each country).

Imagine you were interested in how the life expectancy for each country changes across time.  This data could be reshaped to present this data in a wide form:

In [ ]:
gm_life_exp_wide = gapminder.pivot(index = "country", columns="year", values="lifeExp").reset_index()
gm_life_exp_wide.head(10)

In the above table, the years have now been *pivotted* into individual columns and each cell represents a life expectancy for a given country.  The data table has now become wider (horizontally) than it was previously, where the number of columns is defined by the number of unique values that the year column had in the original form of the data.  If we were to add additional year measurements to this table, it would get wider by adding more year columns.

Here we used the `pandas` `pivot` function to accomplish the long to wide reformatting.  This function takes three main arguments:
* `index`: the column that identifies what each row will be identified by, in this case the country -- each row gives measuremtns for a specific country
* `columns`: the column that identifies the values that will *pivot* into the new column names; this will typically be categorical type column
* `values`: the column that identifies which data values will go into the reshaped table

Also, note the use of the `reset_index` function above.  This function takes the row index produced by the `pivot` function (`country` in this case) and makes it a new column as we can see in the output.  We'll need this column in the example below.

Getting these function arguments right can be tricky in the beginning, so don't get too frustrated.  Expertise will come with practice, and pivotting is such a common task that you'll find plenty of opportunities to pratice.

## Reshaping Wide Data to Long Data

Reformatting data from wide to long format can be accomplished using the `melt` function.  Think of the wide data table *melting* down into a longer table:

In [ ]:
gm_life_exp_long = gm_life_exp_wide.melt(id_vars="country")
gm_life_exp_long.head(10)

This data table is now longer (has more rows), and looks like a subset of the original data table that only has the life expectancy data.

The `melt` function takes all the columns *not specified* in the id_vars argument, and collapses them down to two new columns, one representing the new variable names, and the other representing their corresponding values.  In this particular case, we used the `country` column as the id variable, so the `melt` function collected all the remaining column names, the years, and collapsed them down in to a single column called `year` (this name carried over from the `pivot` step).  All the values in the table, the life expectancies, then got gathered up into a new single column with a default name of `value`.  These new column names can also be defined with additional function arguments:

In [ ]:
gm_life_exp_wide.melt(id_vars="country", var_name = "my_year", value_name = "lifeExp").head(10)

## Combining Separate Tables into One Using `concat`

It's often useful to combine multiple data tables into a single, larger table, for example when you want to merge tables with similar structure that exist across multipe files, or merge together data from two different sources.  Generally speaking, combining tables occurs in two ways: stacking two tables "on top" of each other (combining table rows), or smashing two table together (combining table columns).  

Let's first look at the first case: stacking two tables on top of each other.  Note that in general, the two table should have the exact same column names.  For this example, we'll make two separate tables from our source table:

In [ ]:
us_data = gapminder[gapminder["country"] == "United States"].reset_index(drop = True)
us_data

In [ ]:
japan_data = gapminder[gapminder["country"] == "Japan"].reset_index(drop = True)
japan_data

Next, we can use the `concat` function to *concatenate* these two individual tables together:

In [ ]:
# Combine the USA and Japan data tables by row
# NOTE: the two tables need to be put inside a Python list, []
us_japan_data = pd.concat([us_data, japan_data]).reset_index(drop = True)
us_japan_data

By default, the concat function combines the tables by row, but it's possible to also combine tables by column using the function argument `axis = 1`, (where `1` = "by column", and `0` = "by row").  Concatenating two tables by column "smashes" them together, putting the columns together side by side.  In this case, the column names in the two tables should be different (duplicating column names is a generally bad idea), while the number of rows should be the same.

For this example, we'll create a new table with mock data to add on to the United States data table:

In [ ]:
new_data_dict = {"new_col_1": list(range(0, us_data.shape[0])),
                 "new_col_2": list(range(0, us_data.shape[0] * 2, 2))}
new_data = pd.DataFrame(new_data_dict)
new_data

In [ ]:
pd.concat([us_data, new_data], axis = 1)

## Merging Tables Together Using `merge`

Many types of data problems require combining data from multiple sources.  `concat` can be used to perform simple merging (by rows or columns) as we saw above, but more complex merging can be performed using the `merge` function.  `merge` allows for database-like merging of tables, allowing you to specify exactly how they'd like the two tables to be combined.

As an example, consider our GapMinder data set and that we have data from another source for each country that we want to add to our table.  For simplicity, we'll just work with the United States and Japan table we made above, and work with a data table that has the average temperatures of the two countries for a few years.

In [ ]:
temperature_dict = {"country": ["United States", "United States", "United States", "Japan", "Japan", "Japan"],
                    "year": [1997, 2002, 2007, 1997, 2002, 2007],
                    "avg_temperature_f": [53, 56, 56, 59, 58, 58]}
temperature_df = pd.DataFrame(temperature_dict)
temperature_df

When merging data, it's important to consider which columns correspond to each other between the two tables you want to merge.  In this case, we can see that country and year are common between our tables, and importantly for the merge we are going to do, the values also correspond (i.e. the countries are United States and Japan, and the years are 1997, 2002, and 2007).

In [ ]:
merged_df = pd.merge(us_japan_data, temperature_df, how="left", on=["country", "year"])

There are several things to note about the above `merge`:

- We didn't wrap the two tables to merge into a Python list (like we did for `concat`)
- The `how` argument specifies how we want to merge, in this case, using the left table (i.e. the first one specified in the arguments) as the basis for the merge; all rows from the left table will appear in the output of the merge
- The `on` argument gives the names of the columns we want to use for determining how to do the merge;  in this case we want to merge using the values from the `country` and `year` columns
- Since we joined using the left table (`us_japan_data`), all of the rows in that table appear in the output
- Since the other table only included year values for 1997, 2002, and 2007, we only have temperature values for those years in the output; the rest are missing values (`NaN`)

For a different kind of merge, let's see what happens if we merge using the *right* table as the basis for the merge:

In [ ]:
merged_df_2 = pd.merge(us_japan_data, temperature_df, how="right", on=["country", "year"])
merged_df_2

Now, we only have 3 rows for each country in the output.  Since we used the right table (`temperature_df`) as the basis for the merge, only values that are present in that table remain after the merge.  These row correspond to the three years (1997, 2002, and 2007) that we had data for in the `temperature_df` table.

Merging data is a deep (and complex) topic that goes well beyond the simple examples above, but is also a powerful tool and worth learning about in more detail.

## Main Points

* Reshaping data is the process of reorganizing how data are arranged, and is often needed to make data analysis tasks easier to perform
* Long format data has data structured in a vertical-centric form (i.e. along rows)
* Wide format data has data structured in a horizontal-centric form (i.e. along columns)
* You can convert between long and wide forms using the `pivot` and `melt` functions
* Merging individual tables together can be accomplished with the `concat` and `merge` functions.  Simple concatenation of row or columns is achieved with `concat`.  Use the `merge` function for more complex joining.